In [ ]:
# LSTM RNN Model Using BTC Closing Prices




In [ ]:
# Data Preparation
# Note: The required input format for the LSTM is:
# reshape((X_train.shape[0], X_train.shape[1], 1))

In [68]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [69]:
# Set the random seed
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [70]:
# Load the fear and greed sentiment data for Bitcoin
fng_df=pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
fng_df = fng_df.drop(columns="fng_classification")
fng_df

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42
...,...
2018-02-05,11
2018-02-04,24
2018-02-03,40


In [71]:
# Load the historical closing prices for Bitcoin
historical_df=pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
historical_df=historical_df.sort_index()
historical_df

Date
2018-01-02    14754.129883
2018-01-03    15156.620117
2018-01-04    15180.080078
2018-01-05    16954.779297
2018-01-06    17172.300781
                  ...     
2019-07-25     9882.429688
2019-07-26     9847.450195
2019-07-27     9478.320313
2019-07-28     9531.769531
2019-07-29     9529.889648
Name: Close, Length: 574, dtype: float64

In [72]:
df=fng_df.join(historical_df, how="inner")
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [73]:
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [74]:
def window_data(df, window, feature_col_number, target_col_number):
    X=[]
    y=[]
    for i in range(len(df)-window-1):
        features=df.iloc[i:(i+window), feature_col_number]
        target=df.iloc[(i+window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [75]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window size =10
window=10

# Column index 0 is "fng_value" column
# Column index 1 is the "Close" column
feature_column=1
target_column=1
X, y = window_data(df, window, feature_column, target_column)

In [78]:
# Use 70% of the data for training and the remaineder for testing
split = int(len(X)*0.7)
split

372

In [79]:
X_train=X[:split-1]
X_test=X[split:]
y_train=y[:split - 1]
y_test=y[split:]

In [80]:
from sklearn.preprocessing import MinMaxScaler
x_train_scaler=MinMaxScaler()
x_test_scaler=MinMaxScaler()
y_train_scaler=MinMaxScaler()
y_test_scaler=MinMaxScaler()

x_train_scaler.fit(X_train)
y_train_scaler.fit(y_train)

X_train=x_train_scaler.transform(X_train)
y_train=y_train_scaler.transform(y_train)

In [81]:
x_test_scaler.fit(X_test)
y_test_scaler.fit(y_test)

X_test=x_test_scaler.transform(X_test)
y_test=y_test_scaler.transform(y_test)

In [82]:
# Reshape the features for the model
X_train=X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test=X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

---

In [83]:
# Build and fit a custom LSTM RNN

In [84]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [85]:
model=Sequential()
model.add(LSTM(units=30, return_sequences=True,
    input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=30))
model.add(Dropout(0.2))
model.add(Dense(1))

In [86]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [87]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10, 30)            3840      
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 10, 30)            7320      
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_8 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [88]:
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
371/371 [==============================] - 5s 6ms/step - loss: 0.0835
Epoch 2/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0680
Epoch 3/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0826
Epoch 4/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0797
Epoch 5/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0674
Epoch 6/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0627
Epoch 7/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0599
Epoch 8/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0537
Epoch 9/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0508
Epoch 10/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0495


---

In [ ]:
# Evaluate model performance
# Apply the "inverse_transform" function to the predicted and y_test values

In [89]:
model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 3ms/step - loss: 0.0487


0.04872175678610802

In [90]:
predicted=model.predict(X_test)

In [91]:
# Recover the original prices instead of the scaled version
predicted_prices=y_test_scaler.inverse_transform(predicted)
real_prices=y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

In [92]:
# Create a DataFrame of Real and Predicted values
btc_prices=pd.DataFrame({
    "Real prices": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()}, 
    index = df.index[-len(real_prices): ]) 
btc_prices

,Real prices,Predicted
2019-02-20,3924.239990,4248.814941
2019-02-21,3974.050049,4270.220703
2019-02-22,3937.040039,4296.877441
2019-02-23,3983.530029,4322.457520
2019-02-24,4149.089844,4345.416504
...,...,...
2019-07-25,9772.139648,7328.637695
2019-07-26,9882.429688,7309.071777
2019-07-27,9847.450195,7277.591309
2019-07-28,9478.320313,7253.511719


In [93]:
btc_prices.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)